In [ ]:
from web3 import Web3
from tqdm.notebook import tqdm as tq
import requests
import time
import threading
from collections import Counter

## Initializing the ethereum client

In [ ]:
endpoint = 'wss://mainnet.infura.io/ws/v3/REPLACE_ME'

w3 = Web3(Web3.WebsocketProvider(endpoint))
eth = w3.eth

In [ ]:
contract = {
    'address':w3.toChecksumAddress('0x60f80121c31a0d46b5279700f9df786054aa5ee5'),
    'name':'RARIBLE',
    'firstBlock':10149090
}

In [ ]:
all_entries = []
cur_block = contract['firstBlock']
STEP = 3000

## Fetching all the logs with the specified `Transfer` topic

In [ ]:
topics = [w3.keccak(text='Transfer(address,address,uint256)').hex()]
while cur_block < eth.blockNumber:
    filter_params = {'fromBlock':cur_block,'toBlock':cur_block + STEP,'topics':topics,'address':contract['address']}
    _filter = eth.filter(filter_params)
    entries= _filter.get_all_entries()
    all_entries.extend(entries)
    cur_block += STEP + 1
    print(f'Cur block {cur_block} #entries {len(all_entries)} #step_entries {len(entries)}')
    

In [ ]:
sorted_entries = sorted(all_entries,key=lambda x:(x['blockNumber'],x['transactionIndex']))

## Take a look at one of the entries

In [ ]:
entry = sorted_entries[0]
entry

## Save the entries for later use

In [ ]:
from hexbytes import HexBytes
from web3.datastructures import AttributeDict
def serialize(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, HexBytes):
        return obj.hex()
    
    if isinstance(obj, AttributeDict):
        return dict(obj)

    return obj.__dict__

with open(contract['name']+'_entries.json','w') as f:
    json.dump(sorted_entries,f,default=serialize,indent=2)

In [ ]:
def parse_address(address):
    return address.hex()[-40:]

def parse_entry(entry): # returns source address, destination address, and token id
    return parse_address(entry['topics'][1]), parse_address(entry['topics'][2]),int(entry['topics'][3].hex()[2:],base=16)
    
    
parsed_entries = [parse_entry(entry) for entry in sorted_entries[-1000:]]